# Análise de Velocidade do Vento através da Distribuição de Weibull

# Montando o ambiente para leitura dos dados

In [0]:
#Montando Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
#Instala a biblioteca weibull para análise de distribuição de Weibull
!pip install weibull

In [0]:
path = 'gdrive/My Drive/CCD/'

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import weibull

In [0]:
#Leitura do dataframe contendo as medições de velocidade do vento apenas
vento = pd.read_csv(path + 'Clip/wind_only.csv').drop(columns = 'Unnamed: 0')

In [0]:
#Leitura do dataframe contendo metadados
meta = pd.read_csv(path + 'bdmep_meta.csv')

In [0]:
meta.head()

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1
0,83010,-68.733333,-11.016667,260.00,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
1,82704,-72.666667,-7.633333,170.00,Cruzeiro do Sul,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
2,82915,-67.800000,-9.966667,160.00,Rio Branco,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
3,82807,-70.766667,-8.166667,190.00,Tarauacá,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
4,83098,-36.166667,-10.150000,56.13,Coruripe,Alagoas,AL,America/Maceio,-3,America/Maceio,-3


In [0]:
# Cria novas colunas no dataframe para serem substituidas mais a frente
meta['shape'] = 0
meta['scale'] = 0
meta['vmp'] = 0
meta['vmaxE'] = 0
meta['p_vmp'] = 0
meta['p_vmaxE'] = 0
meta['w_mean'] = 0
meta['w_median'] = 0
meta['mean'] = 0
meta['median'] = 0

# Calculando parâmetros para os dados

In [0]:
def prob_weibull(v, shape, scale):
  
  """
  Função que calcula a respectiva probabilidade de valores menores do que dada
  velocidade do vento (v em m/s) em função dos parâmetros da distribuição de
  Weibull
  """
  
  k = shape
  c = scale
  
  prob = (k/c) * ((v/c) ** (k-1)) * np.exp(-((v/c)**k))
  
  return prob

def most_probable_wspeed(shape, scale):
  
  """
  Função que calcula a velocidade do vento mais provável em função dos 
  parâmetros da distribuição de Weibull
  """
  
  k = shape
  c = scale
  
  aux1 = np.power(((k -1) / k), (1/k))
  
  vmp = c * aux1
  
  return vmp

def most_energy_wspeed(shape, scale):
  
  """
  Função que calcula a velocidade do vento mais energética em função dos 
  parâmetros da distribuição de Weibull
  """
  
  k = shape
  c = scale
  
  vmaxE = c * (( (k + 2) / k) ** (1/k))
  
  return vmaxE

In [0]:
meta.shape

(389, 21)

In [0]:
# O código a seguir irá calcular diversos parâmetros em função da distribuição
# de Weibull para dado conjunto de medições de velocidade do vento

count = 1 # Contador para saber quantas cidades já foram analisadas

for cidade in vento['name'].unique():  
  
  try:
    
    temp = vento[vento['name'] == cidade].ws # Apenas dados de velocidade do vento
    temp = temp[temp != 0].dropna() # Elimina NaNs e valores iguais a zero
    
    # O zero é eliminado pois é necessário para a análise de Weibull
  
    print(f'Dados da cidade #{count}: {cidade} com {len(temp)} observações')
  
    analise = weibull.Analysis(temp) # Prepara os dados para análise
  
    analise.fit(method = 'mle') # Encontra os parâmetros através do método
                                # Maximum Likelihood Estimation (mle)
    shape = analise.beta # Parâmetro de forma (k ou beta)
    scale = analise.eta # Parâmetro de escala (c ou eta)
    
    vmp = most_probable_wspeed(shape, scale)  # Velocidade mais provável
    vmaxE = most_energy_wspeed(shape, scale) # Velocidade mais energética
  
    p_vmp = prob_weibull(vmp, shape, scale) # Probabilidade para vmp
    p_vmaxE = prob_weibull(vmaxE, shape, scale) # Probabilidade para vmaxE
  
    wmean = analise.mean # Valor médio segundo distribuição de Weibull
    wmedian = analise.median # Mediana segunda distribuição de Weibull
    
    mean = np.mean(temp) # Valor médio comum para os dados
    median = np.median(temp) # Mediana comum para os dados
  
    # O código abaixo irá colocar os valores calculados acima nas suas respectivas
    # cidades dentro do dataframe que continha os metadados
       
    meta['shape'] = np.where(meta['name'] == cidade, shape, meta['shape'])
    meta['scale'] = np.where(meta['name'] == cidade, scale, meta['scale'])
    meta['vmp'] = np.where(meta['name'] == cidade, vmp, meta['vmp'])
    meta['vmaxE'] = np.where(meta['name'] == cidade, vmaxE, meta['vmaxE'])
    meta['p_vmp'] = np.where(meta['name'] == cidade, p_vmp, meta['p_vmp'])
    meta['p_vmaxE'] = np.where(meta['name'] == cidade, p_vmaxE, meta['p_vmaxE'])
    meta['w_mean'] = np.where(meta['name'] == cidade, wmean, meta['w_mean'])
    meta['w_median'] = np.where(meta['name'] == cidade, wmedian, meta['w_median'])
    meta['mean'] = np.where(meta['name'] == cidade, mean, meta['mean'])
    meta['median'] = np.where(meta['name'] == cidade, median, meta['median'])
  
  except:
    
    # Quando não há observações de velocidade do vento, substituir
    # os valores que seriam calculados acima por NaNs, os quais serão
    # removidos posteriormente
    
    print(f'Cidade #{count} {cidade} com erros, shape: {temp.shape}.')
    
    meta['shape'] = np.where(meta['name'] == cidade, np.nan, meta['shape'])
    meta['scale'] = np.where(meta['name'] == cidade, np.nan, meta['scale'])
    meta['vmp'] = np.where(meta['name'] == cidade, np.nan, meta['vmp'])
    meta['vmaxE'] = np.where(meta['name'] == cidade, np.nan, meta['vmaxE'])
    meta['p_vmp'] = np.where(meta['name'] == cidade, np.nan, meta['p_vmp'])
    meta['p_vmaxE'] = np.where(meta['name'] == cidade, np.nan, meta['p_vmaxE'])
    meta['w_mean'] = np.where(meta['name'] == cidade, np.nan, meta['w_mean'])
    meta['w_median'] = np.where(meta['name'] == cidade, np.nan, meta['w_median'])
    meta['mean'] = np.where(meta['name'] == cidade, np.nan, meta['mean'])
    meta['median'] = np.where(meta['name'] == cidade, np.nan, meta['median'])
  
  count += 1 # Contador para saber quantas cidades já foram analisadas

Dados da cidade #1: Brasiléia com 1274 observações
Dados da cidade #2: Cruzeiro do Sul com 8149 observações
Dados da cidade #3: Rio Branco com 14628 observações
Dados da cidade #4: Tarauacá com 12974 observações
Dados da cidade #5: Coruripe com 4281 observações
Dados da cidade #6: Maceió com 11638 observações
Dados da cidade #7: Mata Grande com 3652 observações
Dados da cidade #8: Palmeira dos Índios com 12476 observações
Dados da cidade #9: Pão de Acuçar com 9255 observações
Dados da cidade #10: Porto de Pedras com 11947 observações
Dados da cidade #11: Barcelos com 18261 observações
Dados da cidade #12: Benjamin Constant com 16365 observações
Dados da cidade #13: Carauari com 6225 observações
Dados da cidade #14: Coari com 16116 observações
Dados da cidade #15: Codajás com 13936 observações
Dados da cidade #16: Eirunepé com 3556 observações
Dados da cidade #17: Fonte Boa com 16748 observações
Dados da cidade #18: Humaitá com 5624 observações
Dados da cidade #19: Itacoatiara com 14022

Dados da cidade #81: Itarana com 1 observações
Cidade #81 Itarana com erros, shape: (1,).


/usr/local/lib/python3.6/dist-packages/weibull/weibull.py:160: RuntimeWarning: invalid value encountered in double_scalars
  / (dtf_all.size - 1)) ** -0.5


Dados da cidade #82: Linhares com 10388 observações


Dados da cidade #83: Muniz Freire com 7 observações
Dados da cidade #84: São Mateus com 6796 observações
Dados da cidade #85: Venda Nova com 618 observações
Dados da cidade #86: Vitória com 13757 observações
Dados da cidade #87: Aragarças com 13394 observações
Dados da cidade #88: Catalão com 20036 observações
Dados da cidade #89: Formosa com 16574 observações
Dados da cidade #90: Goianésia com 5406 observações
Dados da cidade #91: Goiânia com 16731 observações
Dados da cidade #92: Goiás com 18268 observações
Dados da cidade #93: Ipameri com 14678 observações
Dados da cidade #94: Jataí com 8598 observações
Dados da cidade #95: Mineiros com 3596 observações
Dados da cidade #96: Pirenópolis com 14673 observações
Dados da cidade #97: Posse com 12134 observações
Dados da cidade #98: Rio Verde com 12700 observações
Dados da cidade #99: Alto Parnaíba com 12448 observações
Dados da cidade #100: Bacabal com 12233 observações
Dados da cidade #101: Balsas com 13064 observações
Dados da cidade #1

Dados da cidade #257: Alto da Boa Vista com 2 observações
Dados da cidade #258: Angra dos Reis com 4131 observações
Dados da cidade #259: Araras com 5301 observações
Dados da cidade #260: Avelar com 9135 observações
Dados da cidade #261: Bangu com 1641 observações
Dados da cidade #262: Barreirinha com 5944 observações
Dados da cidade #263: Cabo Frio (Alcalis) com 5863 observações
Dados da cidade #264: Campos  com 15842 observações
Dados da cidade #265: Carmo com 4071 observações
Dados da cidade #266: Cordeiro com 921 observações
Dados da cidade #267: Ecologia Agrícola com 7930 observações
Dados da cidade #268: Engenho de Dentro com 435 observações
Dados da cidade #269: Gra. Juriti (Petrópolis) com 0 observações
Cidade #269 Gra. Juriti (Petrópolis) com erros, shape: (0,).
Dados da cidade #270: Iguaba Grande com 3137 observações
Dados da cidade #271: Ilha Guaíba com 4603 observações
Dados da cidade #272: Itaperuna com 8711 observações
Dados da cidade #273: Jacarepaguá com 1687 observaçõe

/usr/local/lib/python3.6/dist-packages/weibull/weibull.py:165: RuntimeWarning: overflow encountered in power
  b = np.sum(dtf_all ** shape)
/usr/local/lib/python3.6/dist-packages/weibull/weibull.py:166: RuntimeWarning: overflow encountered in power
  c = np.sum((dtf_all ** shape) * np.log(dtf_all))
/usr/local/lib/python3.6/dist-packages/weibull/weibull.py:167: RuntimeWarning: overflow encountered in power
  h = np.sum((dtf_all ** shape) * (np.log(dtf_all)) ** 2)
/usr/local/lib/python3.6/dist-packages/weibull/weibull.py:169: RuntimeWarning: invalid value encountered in double_scalars
  shape = shape + (a + (1.0 / shape) - (c / b)) / ((1.0 / shape ** 2) + ((b * h) - c ** 2) / b ** 2)


Cidade #368 Santa Rita com erros, shape: (6267,).
Dados da cidade #369: Santos com 9151 observações
Dados da cidade #370: São Carlos com 19666 observações
Dados da cidade #371: São José dos Campos com 5667 observações
Dados da cidade #372: São Paulo - Horto Florestal com 6905 observações
Dados da cidade #373: São Paulo (Mir. de Santana) com 20681 observações
Dados da cidade #374: São Simão com 15703 observações
Dados da cidade #375: Sertãozinho com 3147 observações
Dados da cidade #376: Sorocaba com 11393 observações
Dados da cidade #377: Tamoio com 3524 observações
Dados da cidade #378: Taubaté com 7928 observações
Dados da cidade #379: Tremembé com 5554 observações
Dados da cidade #380: Ubatuba com 14352 observações
Dados da cidade #381: Usina Junqueira com 3570 observações
Dados da cidade #382: Votuporanga com 7933 observações
Dados da cidade #383: Araguaina com 12347 observações
Dados da cidade #384: Paranã com 9026 observações
Dados da cidade #385: Pedro Afonso com 13058 observaçõ

# Identificando e consertando possíveis erros

In [0]:
len(meta['name'].unique()) # Número de cidades com nomes únicos

388

In [0]:
meta.shape # Formato do dataframe, 389 é diferente de 388, logo há uma cidade com nome repetido

(389, 21)

In [0]:
meta.name[meta['name'].duplicated(keep = False)] # Descobre nomes repetidos

250    Palmas
383    Palmas
Name: name, dtype: object

In [0]:
meta[meta['name'] == 'Palmas'] # Há duas cidades com nome Palmas, é preciso recalcular os seus parâmetros

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1,shape,scale,vmp,vmaxE,p_vmp,p_vmaxE,w_mean,w_median,mean,median
250,83860,-51.983333,-26.483333,1090.52,Palmas,Paraná,PR,America/Sao_Paulo,-3,America/Sao_Paulo,-3,1.54496,1.636089,0.37354,2.429874,0.381242,0.185596,1.471999,1.290561,1.463766,1.333333
383,83033,-48.300000,-10.183333,280.00,Palmas,Tocantins,TO,America/Araguaina,-3,America/Araguaina,-3,1.54496,1.636089,0.37354,2.429874,0.381242,0.185596,1.471999,1.290561,1.463766,1.333333


In [0]:
meta.loc[250,:] # Palmas/PR

id                          83860
lon                      -51.9833
lat                      -26.4833
alt                       1090.52
name                       Palmas
state                      Paraná
uf                             PR
time_zone       America/Sao_Paulo
offset_utc                     -3
time_zone.1     America/Sao_Paulo
offset_utc.1                   -3
shape                     1.54496
scale                     1.63609
vmp                       0.37354
vmaxE                     2.42987
p_vmp                    0.381242
p_vmaxE                  0.185596
w_mean                      1.472
w_median                  1.29056
mean                      1.46377
median                    1.33333
Name: 250, dtype: object

In [0]:
# Calculando os parâmetros para Palmas/PR

temp = vento[(vento['name'] == 'Palmas') & (vento['uf'] == 'PR')].ws
temp = temp[temp != 0].dropna()

print(temp.shape)

analise = weibull.Analysis(temp)
  
analise.fit(method = 'mle')
    
shape = analise.beta
scale = analise.eta
  
print(f'beta: {shape: .02f}')
print(f'eta: {scale: .02f}')
    
vmp = most_probable_wspeed(shape, scale) 
vmaxE = most_energy_wspeed(shape, scale)
  
p_vmp = prob_weibull(vmp, shape, scale)
p_vmaxE = prob_weibull(vmaxE, shape, scale)
  
wmean = analise.mean
wmedian = analise.median
    
mean = np.mean(temp)
median = np.median(temp)

meta.loc[250,'shape'] = shape
meta.loc[250,'scale'] = scale
meta.loc[250,'vmp'] = vmp
meta.loc[250,'vmaxE'] = vmaxE
meta.loc[250,'p_vmp'] = p_vmp
meta.loc[250,'p_vmaxE'] = p_vmaxE
meta.loc[250,'w_mean'] = wmean
meta.loc[250,'w_median'] = wmedian
meta.loc[250,'mean'] = mean
meta.loc[250,'median'] = median

(3478,)
beta:  1.60
eta:  1.57


In [0]:
meta.loc[383,:] #Palmas/TO

id                          83033
lon                         -48.3
lat                      -10.1833
alt                           280
name                       Palmas
state                   Tocantins
uf                             TO
time_zone       America/Araguaina
offset_utc                     -3
time_zone.1     America/Araguaina
offset_utc.1                   -3
shape                     1.54496
scale                     1.63609
vmp                       0.37354
vmaxE                     2.42987
p_vmp                    0.381242
p_vmaxE                  0.185596
w_mean                      1.472
w_median                  1.29056
mean                      1.46377
median                    1.33333
Name: 383, dtype: object

In [0]:
# Calculando os parâmetros para Palmas/TO

temp = vento[(vento['name'] == 'Palmas') & (vento['uf'] == 'TO')].ws
temp = temp[temp != 0].dropna()

print(temp.shape)

analise = weibull.Analysis(temp)
  
analise.fit(method = 'mle')
    
shape = analise.beta
scale = analise.eta
  
print(f'beta: {shape: .02f}')
print(f'eta: {scale: .02f}')
    
vmp = most_probable_wspeed(shape, scale) 
vmaxE = most_energy_wspeed(shape, scale)
  
p_vmp = prob_weibull(vmp, shape, scale)
p_vmaxE = prob_weibull(vmaxE, shape, scale)
  
wmean = analise.mean
wmedian = analise.median
    
mean = np.mean(temp)
median = np.median(temp)

meta.loc[383,'shape'] = shape
meta.loc[383,'scale'] = scale
meta.loc[383,'vmp'] = vmp
meta.loc[383,'vmaxE'] = vmaxE
meta.loc[383,'p_vmp'] = p_vmp
meta.loc[383,'p_vmaxE'] = p_vmaxE
meta.loc[383,'w_mean'] = wmean
meta.loc[383,'w_median'] = wmedian
meta.loc[383,'mean'] = mean
meta.loc[383,'median'] = median

(7801,)
beta:  1.52
eta:  1.67


In [0]:
meta[meta['name'] == 'Palmas'] # Verificando se os parâmetros foram alterados de fato

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1,shape,scale,vmp,vmaxE,p_vmp,p_vmaxE,w_mean,w_median,mean,median
250,83860,-51.983333,-26.483333,1090.52,Palmas,Paraná,PR,America/Sao_Paulo,-3,America/Sao_Paulo,-3,1.602136,1.568519,0.367947,2.201158,0.386804,0.224086,1.406121,1.247780,1.396674,1.333333
383,83033,-48.300000,-10.183333,280.00,Palmas,Tocantins,TO,America/Araguaina,-3,America/Araguaina,-3,1.524961,1.666206,0.376130,2.525606,0.377850,0.172752,1.501252,1.310235,1.493678,1.266667


In [0]:
# Remover linhas com poucas observações < 10
# cidades com poucas observações geraram valores anômalos  ['Alto da Boa Vista', 'Muniz Freire']
# é preciso removê-las

meta[meta['name'] == 'Alto da Boa Vista']

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1,shape,scale,vmp,vmaxE,p_vmp,p_vmaxE,w_mean,w_median,mean,median
256,83007,-43.266667,-22.95,347.09,Alto da Boa Vista,Rio de Janeiro,RJ,America/Sao_Paulo,-3,America/Sao_Paulo,-3,113.965261,9.549067,0.083054,0.08526,1.997957e-232,3.858955e-231,9.501424,9.518406,9.5,9.5


In [0]:
meta[meta['name'] == 'Muniz Freire']

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1,shape,scale,vmp,vmaxE,p_vmp,p_vmaxE,w_mean,w_median,mean,median
82,83013,-41.4,-20.45,530.0,Muniz Freire,Espírito Santo,ES,America/Sao_Paulo,-3,America/Sao_Paulo,-3,5.532754,3.681266,0.545101,0.905875,0.000261,0.00261,3.399695,3.445305,3.380606,3.60108


In [0]:
meta.drop([256, 82], axis = 0, inplace = True) # Removendo Alto da Boa Vista e Muniz Freire

In [0]:
meta.name[meta['shape'].isna()] # Identificando cidades sem observações de velocidade do vento

75                   Alfredo Chaves
76                          Aracruz
77                    Boa Esperança
79                       Ecoporanga
80                          Itarana
189                Rosário do Oeste
228    Alto Longa (Piloto do Longa)
268        Gra. Juriti (Petrópolis)
367                      Santa Rita
Name: name, dtype: object

In [0]:
meta.dropna(axis = 0, inplace = True) # Removendo as cidades acima

In [0]:
meta.shape # Verificando forma do dataframe

(378, 21)

In [0]:
meta.reset_index(drop = True, inplace = True) # Resetando índice

In [0]:
meta.to_excel(path + 'weibulldatabase.xls') # Salva os dados para Excel
meta.to_csv(path + 'weibulldatabase.csv') # Salva os dados para csv